In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import ipywidgets as widgets
from pprint import pprint
import json

In [2]:
# Import API key
from config import weather_api_key
from config import g_key
gmaps.configure(api_key=g_key)

# Store Part 1 Results into DF

In [3]:
#Store Part I results into DataFrame by loading the csv exported in Part I to a DataFrame
vacay_df = pd.read_csv("../output_data/cities.csv")
#vacay_df["Hotel Name"]=""
vacay_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,haines junction,60.7522,-137.5108,21.20,93,12,1.14,CA,2021-04-24 12:39:46
1,port alfred,-33.5906,26.8910,74.77,57,15,14.25,ZA,2021-04-24 12:39:46
2,port hardy,50.6996,-127.4199,44.60,100,90,2.30,CA,2021-04-24 12:39:47
3,ahuimanu,21.4447,-157.8378,73.40,73,75,10.36,US,2021-04-24 12:39:47
4,mao,39.8885,4.2658,66.20,59,20,6.91,ES,2021-04-24 12:39:47


# Humidity Heatmap

In [4]:
locations = vacay_df[["Lat","Lng"]]
weights = vacay_df["Humidity"]

In [12]:
#Humidity Heatmap THAT DISPLAYS HUMIDITY FOR EVERY CITY FROM PART 1
#Configure gmaps.
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.
fig1 = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, max_intensity=150, point_radius = 4)
fig1.add_layer(heat_layer)
plt.savefig("../output_data/Map1")
fig1

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

# Create New DF Fitting Weather Criteria

In [6]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.
#MAx temp < 80 but > 70, wind speed < 10 mph, zero cloudiness - DROP ANY ROWS THAT DON'T MEET ALL 3 CONDITIONS 
ideal_df = vacay_df.loc[(vacay_df["Max Temp"] <80) & (vacay_df["Max Temp"] >70) & (vacay_df["Wind Speed"] <10) & (vacay_df["Cloudiness"] ==0)]
#ideal_df.describe()
ideal_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,arraial do cabo,-22.9661,-42.0278,77.0,78,0,9.22,BR,2021-04-24 12:39:49
49,vilhena,-12.7406,-60.1458,73.4,94,0,8.05,BR,2021-04-24 12:39:59
74,east london,-33.0153,27.9116,73.4,56,0,9.22,ZA,2021-04-24 12:40:08
172,semme,46.1963,1.5007,71.6,26,0,9.22,FR,2021-04-24 12:40:34
178,carnarvon,-24.8667,113.6333,73.4,78,0,8.05,AU,2021-04-24 12:40:35


# Hotel Map

In [7]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Store into variable named hotel_df.
#"TEN OR LESS PINS FOR ALL CITIES IN THE DF"
# "PINS ARE ALL CLICKABLE TO DISPLAY CITY, COUNTRY, AND HOTEL NAME"

In [8]:
Hlat = []
Hlon = []
Hcountry = []
Hcity=[]
Hname = []
Haddress = []

## Dear Grader: > 10 hotels here.  If i change target radius to less than 6500 it creates errors. 

In [9]:
for index, row in ideal_df.iterrows():
    try:
        lat = row["Lat"]
        lng = row["Lng"]
        target_coordinates =  f"{lat},{lng}"
        target_search = "hotel"
        target_radius = 6500
        #target_type = "hotel"
        
        params = {
            "location": target_coordinates,
            "keyword": target_search,
            "radius": target_radius,
            #"type": target_type,
            "key": g_key
            }   
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        names_etc = requests.get(base_url, params=params).json()
        #names_etc = name_etc.json()
        #pprint(names_etc)
        Hname.append(names_etc["results"][0]["name"])
        Hlat.append(names_etc["results"][0]["geometry"]["location"]["lat"])
        Hlon.append(names_etc["results"][0]["geometry"]["location"]["lng"])
        Hcountry.append(row["Country"])
        Hcity.append(row["City"])
        print(f"The hotel {names_etc['results'][0]['name']} was found with the lat of {names_etc['results'][0]['geometry']['location']['lat']} and lon of {names_etc['results'][0]['geometry']['location']['lng']}, appending stats.")
    except KeyError:
        print("Sorry, that hotel was not found.")
        pass

The hotel Ocean View Hotel was found with the lat of -22.9587565 and lon of -42.0259149, appending stats.
The hotel Hotel Mirage was found with the lat of -12.7406969 and lon of -60.1406974, appending stats.
The hotel Blue Lagoon Hotel was found with the lat of -32.9834715 and lon of 27.9510976, appending stats.
The hotel Inter-Hotel Alexia was found with the lat of 46.2307752 and lon of 1.5064403, appending stats.
The hotel Hospitality Carnarvon was found with the lat of -24.8923413 and lon of 113.6550343, appending stats.
The hotel Village Toue was found with the lat of 46.4757857 and lon of 3.9739872, appending stats.
The hotel Hotel Goldene Rose Superior was found with the lat of 46.6286737 and lon of 10.7753199, appending stats.
The hotel Pousada Kactus was found with the lat of -21.665366 and lon of -41.023183, appending stats.
The hotel Kempinski Hotel Xiamen was found with the lat of 24.472597 and lon of 118.100434, appending stats.
The hotel Hotel Senac Ilha do Boi was found w

In [10]:
hotel_df = pd.DataFrame({"Hotel Name": Hname, 
                       "Hotel Lat" : Hlat,
                       "Hotel Lng" : Hlon,
                       "Hotel Country": Hcountry,
                       "Hotel City" : Hcity
                        })
#all_df["Date"] = pd.to_datetime(all_df["Date"], unit ='s')
hotel_df.head(20)

,Hotel Name,Hotel Lat,Hotel Lng,Hotel Country,Hotel City
0,Ocean View Hotel,-22.958756,-42.025915,BR,arraial do cabo
1,Hotel Mirage,-12.740697,-60.140697,BR,vilhena
2,Blue Lagoon Hotel,-32.983472,27.951098,ZA,east london
3,Inter-Hotel Alexia,46.230775,1.506440,FR,semme
4,Hospitality Carnarvon,-24.892341,113.655034,AU,carnarvon
5,Village Toue,46.475786,3.973987,FR,saint-francois
6,Hotel Goldene Rose Superior,46.628674,10.775320,IT,lasa
7,Pousada Kactus,-21.665366,-41.023183,BR,sao joao da barra
8,Kempinski Hotel Xiamen,24.472597,118.100434,CN,xiamen
9,Hotel Senac Ilha do Boi,-20.310606,-40.282680,BR,vila velha


In [ ]:
############ NOTE: Do not change any of the code in this cell###############
####### THIS IS HTML TEMPLATE################

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure